In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

In [ ]:
feature = pd.read_csv('../input/store-sales-prediction/featuresdata.csv')
salesdata = pd.read_csv('../input/store-sales-prediction/salesdata.csv')
storesdata = pd.read_csv('../input/store-sales-prediction/storesdata.csv')

In [ ]:
display(feature.head())
display(salesdata.head())
display(storesdata.head())

In [ ]:
feature.info()

In [ ]:
df = feature.isnull().sum()

In [ ]:
round(100*df/len(feature.index),2)

In [ ]:
feature.describe()

In [ ]:
feature.IsHoliday.value_counts()

In [ ]:
feature.Date

In [ ]:
feature['DateTime'] = pd.to_datetime(feature['Date'])
feature['DateTime']

In [ ]:
salesdata.info()

In [ ]:
df_sales = salesdata.isnull().sum()
df_sales

In [ ]:
round(100*df_sales/len(salesdata.index),2)

In [ ]:
salesdata.describe()

In [ ]:
salesdata['Date']
salesdata['year'] = pd.DatetimeIndex(salesdata['Date']).year
salesdata['month'] = pd.DatetimeIndex(salesdata['Date']).month
salesdata['day'] = pd.DatetimeIndex(salesdata['Date']).day

display(salesdata['day'].head(10), salesdata['month'].head(10), salesdata['year'].head(10))

In [ ]:
#salesdata.groupby('year').mean()
salesdata.groupby('year').mean().Weekly_Sales

In [ ]:
salesdata.groupby('month').mean().Weekly_Sales

In [ ]:
salesdata.groupby('day').mean().Weekly_Sales

In [ ]:
storesdata.info()

In [ ]:
df_stores = storesdata.isnull().sum()
df_stores

In [ ]:
round(100*df_stores/len(storesdata.index),2)

In [ ]:
storesdata.describe()

In [ ]:
#testing
# salesdata_2 = pd.read_csv('../input/store-sales-prediction/salesdata.csv')
# storesdata_2 = pd.read_csv('../input/store-sales-prediction/storesdata.csv')

merged_sheet_1=salesdata.merge(storesdata,how='left',on='Store')
display(merged_sheet_1.head())

In [ ]:
display(salesdata.head())

In [ ]:
merged_sheet_1.shape

In [ ]:
salesdata.shape

In [ ]:
merged_sheet_2 = merged_sheet_1.merge(feature,how='left',on=['Store','Date', 'IsHoliday'])
display(merged_sheet_2.head(5))

In [ ]:
merged_sheet_2.shape

In [ ]:
sheet_dropped_na=merged_sheet_2.dropna(subset=['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'])
display(sheet_dropped_na.head())

In [ ]:
merged_sheet_2.shape

In [ ]:
sheet_dropped_na.shape

In [ ]:
Max_sales_store=sheet_dropped_na.groupby('Store').max().Weekly_Sales
Max_sales_store.plot.bar()

In [ ]:
Max_sales_dept=sheet_dropped_na.groupby('Dept').max().Weekly_Sales
Max_sales_dept.plot.bar()

In [ ]:
sheet_dropped_na.info()

In [ ]:
dummies_type = pd.get_dummies(sheet_dropped_na['Type'])
dummies_type.head()

In [ ]:
dummies_type.shape

In [ ]:
dummies_holiday = pd.get_dummies(sheet_dropped_na['IsHoliday'])
dummies_holiday.head()

In [ ]:
dummies_holiday.shape

In [ ]:
dummies_date = pd.get_dummies(sheet_dropped_na['Date'])
dummies_date.head()

In [ ]:
sheet_with_dummies=pd.concat([sheet_dropped_na,dummies_type],axis=1)
sheet_with_dummies.head()

In [ ]:
sheet_with_dummies=pd.concat([sheet_with_dummies,dummies_holiday],axis=1)
sheet_with_dummies.head()

In [ ]:
sheet_date_dropped=sheet_with_dummies.drop(columns=['Date'])
sheet_date_dropped.head()

In [ ]:
sheet_date_dropped.shape

In [ ]:
sheet_date_dropped=sheet_date_dropped[sheet_date_dropped['Weekly_Sales']>100]
sheet_date_dropped.shape

In [ ]:
y = sheet_date_dropped['Weekly_Sales']
y

In [ ]:
X= sheet_date_dropped.loc[:,sheet_date_dropped.columns!='Weekly_Sales']
X

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
y_train.shape

In [ ]:
X_train_scaled = X_train.drop(columns=['IsHoliday','Type','day','month','year','Store','Dept','DateTime'])
# X_train_scaled.head()
# X_train_scaled.shape
# X_train_scaled.info()
scaler = StandardScaler()
X_scaled_features=scaler.fit_transform(X_train_scaled)


In [ ]:
X_scaled_features_df = pd.DataFrame(X_scaled_features, index=X_train_scaled.index, columns=X_train_scaled.columns)
X_scaled_features_df.head()

In [ ]:
X_test_scaled = X_test.drop(columns=['IsHoliday','Type','day','month','year','Store','Dept','DateTime'])
# scaler = StandardScaler()
X_test_scaled_features=scaler.fit_transform(X_test_scaled)
X_test_scaled_features_df = pd.DataFrame(X_test_scaled_features, index=X_test_scaled.index, columns=X_test_scaled.columns)
X_test_scaled_features_df.head()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
X_train_final = X_scaled_features_df.join(X_train["IsHoliday"])
X_train_final = X_train_final.join(X_train["Type"])
X_train_final = X_train_final.join(X_train["day"])
X_train_final = X_train_final.join(X_train["month"])
X_train_final = X_train_final.join(X_train["year"])
X_train_final = X_train_final.join(X_train["Store"])
X_train_final = X_train_final.join(X_train["Dept"])
X_train_final = X_train_final.join(X_train["DateTime"])
X_train_final = X_train_final.drop(columns = ['A', 'B', 'C', True, False, 'DateTime'])
X_train_final = pd.get_dummies(X_train_final, prefix = ['Type','IsHoliday'], columns = ['Type', 'IsHoliday'])
X_train_final.head()                           

In [ ]:
X_test_final = X_test_scaled_features_df.join(X_test["IsHoliday"])
X_test_final = X_test_final.join(X_test["Type"])
X_test_final = X_test_final.join(X_test["day"])
X_test_final = X_test_final.join(X_test["month"])
X_test_final = X_test_final.join(X_test["year"])
X_test_final = X_test_final.join(X_test["Store"])
X_test_final = X_test_final.join(X_test["Dept"])
X_test_final = X_test_final.join(X_test["DateTime"])
X_test_final = X_test_final.drop(columns = ['A', 'B', 'C', True, False, 'DateTime'])
X_test_final = pd.get_dummies(X_test_final, prefix = ['Type','IsHoliday'], columns = ['Type', 'IsHoliday'])
pd.set_option('display.max_columns', None)
X_test_final.head()

In [ ]:
X_train_final.shape

In [ ]:
y_train.shape

In [ ]:
import tensorflow
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=20, kernel_initializer='normal', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
history=model.fit(X_train_final, y_train, epochs=50, batch_size=256, verbose=1, validation_split=0.1)

In [ ]:
preds = model.predict(X_test_final)

In [ ]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_test, preds))

In [ ]:
preds.shape
y_test.shape